In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import *
# import utils; reload(utils)
# from utils import plots
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.preprocessing import image
from keras import backend as K
#from theano import tensor as K
from scipy import ndimage

import numpy as np
import os
import PIL
from PIL import Image

Using TensorFlow backend.


In [2]:
os.getcwd()

'/nfsroot/data/home/s_jaysetty/Insofe_Cluster/hand_gesture'

In [3]:
# VGG model definition
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

# Load the VGG weigths

# path_weights_th = '/Users/Apple/Downloads/Insofe/AI/Transfer_Learning/20170916_Batch_29_CSE7321c_Transfer_Learning/vgg16_weights_th_dim_ordering_th_kernels.h5'
path_weights_tf = '/nfsroot/data/home/s_jaysetty/Insofe_Cluster/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
# path_weights_tf = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
#theano
model = VGG_16(path_weights_tf)

#tensorflow
# model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')

/nfsroot/data/home/s_jaysetty/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  """
/nfsroot/data/home/s_jaysetty/.local/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys
/nfsroot/data/home/s_jaysetty/.local/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/nfsroot/data/home/s_jaysetty/.local/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  del sys.path[0]
/nfsroot/data/home/s_jaysetty/.local/lib/python2.7/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 

In [4]:
size = 224

def read_image(f):
    try:
        im = Image.open(f)
        im = im.resize((size, size), PIL.Image.NEAREST)
        im = np.asarray(im, dtype='float64')
        if len(im.shape)==2:
            im = im.reshape(im.shape[0],im.shape[1],1)
            im = im.repeat(3,axis=2)
        if im.shape[2]!=3:
            im = im[:,:,:3]
        im[:,:,0] -= 103.939
        im[:,:,1] -= 116.779
        im[:,:,2] -= 123.68
        im = im.reshape(1,224,224,3)
        return(im)
    except ValueError:
        print(f) #print the shape of im if it doesn't work correctly, don't reference this apriori

In [5]:
# Basic Check of read_image
path = '/home/s_jaysetty/Insofe_Cluster/hand_gesture/Triesch/'
im = read_image(path + 'kbrauei1.pgm')
# im = read_image('/home/s_jaysetty/Insofe_Cluster/hand_gesture/Triesch/bfritza2.pgm')
print(im.shape)
# model.layers[32]

(1, 224, 224, 3)


In [6]:
# with a Sequential model, get output of only 33rd layer
get_33rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[32].output])
layer_output = get_33rd_layer_output([im])[0]
#print(layer_output.shape)

In [7]:
img_files = []
path = '/home/s_jaysetty/Insofe_Cluster/hand_gesture/Triesch'
img_files.extend([files for files in os.listdir(path) if files.endswith('.pgm')])
print(len(img_files))
print(img_files[:10])

718
['bfritza1.pgm', 'bfritza2.pgm', 'bfritza3.pgm', 'bfritzb1.pgm', 'bfritzb2.pgm', 'bfritzb3.pgm', 'bfritzc1.pgm', 'bfritzc2.pgm', 'bfritzc3.pgm', 'bfritzd1.pgm']


In [8]:
#Shuffle the Triesch images
n_files = len(img_files)
print('Total number of images:', n_files)

#train_size = 2000
#for demo
train_size = 10
sample_size = max(n_files,train_size)
print('Sample images taken:', sample_size)
sample = [img_files[i] for i in np.random.choice(len(img_files),size=sample_size, replace=False)]
print(sample[:10])
print(len(sample))

('Total number of images:', 718)
('Sample images taken:', 718)
['mkefalb3.pgm', 'jwieghc1.pgm', 'kbrauei1.pgm', 'gpeterl1.pgm', 'pleuchl3.pgm', 'haloosd1.pgm', 'tkerscy3.pgm', 'mkefalh1.pgm', 'tkerscc2.pgm', 'mrinnel1.pgm']
718


In [9]:
# Add Triesch images to train + val
i=0
x_train = []
y_train = []
path = '/home/s_jaysetty/Insofe_Cluster/hand_gesture/Triesch/'
print('Reading Triesch train images ...')
for file in sample[:574]:
    if(i%100==0):
        print("Read {} images".format(i))
    im = read_image(path+file)
#         im = read_image('/home/s_jaysetty/Insofe_Cluster/hand_gesture/Triesch/'+file)
    temp = get_33rd_layer_output([im])[0]
    x_train.append(temp[0])
    y_train.extend([0 if file[6]=='a' else
          1 if file[6]=='b' else 
          2 if file[6]=='c' else 
          3 if file[6]=='d' else
          4 if file[6]=='g' else
          5 if file[6]=='h' else
          6 if file[6]=='i' else
          7 if file[6]=='l' else
          8 if file[6]=='v' else 9])
        
    i+=1
print('Successfully read {} Triesch train images ...'.format(i))

Reading Triesch train images ...
Read 0 images
Read 100 images
Read 200 images
Read 300 images
Read 400 images
Read 500 images
Successfully read 574 Triesch train images ...


In [10]:
# Add Triesch images to train + val
j=0
x_val = []
y_val = []
path = '/home/s_jaysetty/Insofe_Cluster/hand_gesture/Triesch/'
print('Reading Triesch val images ...')
for file in sample[574:]:
    if(i%100==0):
        print("Read {} images".format(i))
    im = read_image(path+file)
    temp = get_33rd_layer_output([im])[0]
    x_val.append(temp[0])
    y_val.extend([0 if file[6]=='a' else
          1 if file[6]=='b' else 
          2 if file[6]=='c' else 
          3 if file[6]=='d' else
          4 if file[6]=='g' else
          5 if file[6]=='h' else
          6 if file[6]=='i' else
          7 if file[6]=='l' else
          8 if file[6]=='v' else 9])
        
    j+=1
print('Successfully read {} Triesch val images ...'.format(j))

Reading Triesch val images ...
Successfully read 144 Triesch val images ...


In [24]:
# Add the MLP Layer
mlp = Sequential()
mlp.add(Dense(32, input_dim=4096, activation='relu'))
mlp.add(Dropout(0.4))
# mlp.add(Dense(128, activation='relu'))
# mlp.add(Dropout(0.4))
mlp.add(Dense(10, activation='sigmoid'))#try dense as 1 and sigmoid
mlp.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 32)                131104    
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                330       
Total params: 131,434
Trainable params: 131,434
Non-trainable params: 0
_________________________________________________________________


In [33]:
#adam = Adam(lr=0.1)
# adam = keras.optimizers.adam()
# mlp.compile(optimizer='Adam', 
#             loss='categorical_crossentropy', 
#             metrics=['acc'])
sgd = SGD(lr=0.04, decay=1e-6, momentum=0.6, nesterov=True)
mlp.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy', recall])

mlp.fit(np.array(x_train), to_categorical(y_train), nb_epoch=3, validation_data=(np.array(x_val), to_categorical(y_val)))


Train on 574 samples, validate on 144 samples
Epoch 1/3
574/574 [==============================] - 0s - loss: 0.2026 - acc: 0.9077 - val_loss: 0.7838 - val_acc: 0.9167
Epoch 2/3
574/574 [==============================] - 0s - loss: 0.1714 - acc: 0.9199 - val_loss: 0.7787 - val_acc: 0.9167
Epoch 3/3
574/574 [==============================] - 0s - loss: 0.1097 - acc: 0.9495 - val_loss: 0.8441 - val_acc: 0.8889
